In [1]:
# 파일 리드
import pandas as pd
d_df_new = pd.read_csv('b-prep.csv', sep=',')

# 선두에서 5행까지 표시
d_df_new.head()

,age,number_yoy,member,period,budget,y,Hawaii,Ho Chi Minh,Hong Kong,Jeju,...,South America,autumn,fall,spring,summer,winter,bicycle,plane,ship,walking
0,20,8,1.0,6,350000,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,20,10,2.0,5,350000,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,20,17,1.0,4,350000,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,20,17,2.0,1,300000,1,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
4,20,19,1.0,2,310000,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0


In [4]:
# Numpy와 RandomUnderSampler의 로드
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# 데이터 세트를 설명변수와 목적변수로 분할
X = np.array(d_df_new.drop('y', axis=1))
Y = np.array(d_df_new['y'])

# y가 1과 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

# y가 1인 데이터의 건수에 맞춰서 다운사이징
sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# y가 1과 0인 데이터 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

1854 1017
1017 1017


In [3]:
# Numpy와 RandomOverSampler의 로드
import numpy as np
from imblearn.over_sampling import RandomOverSampler

# 데이터 세트를 설명변수와 목적변수로 분할
X = np.array(d_df_new.drop('y', axis=1))
Y = np.array(d_df_new['y'])

# y가 1과 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

# y가 1인 데이터의 건수에 맞춰서 다운사이징
sampler = RandomOverSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# y가 1과 0인 데이터 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

1854 1017
1854 1854


In [5]:
# Scikit-learn의 로드
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

# K-Fold 교차검증
kf = KFold(n_splits=10, shuffle=True)
# 모델의 정밀도를 보존할 준비
scores = []
# 데이터를 셔플하고, 훈련데이터와 테스트데이트로 분할
for train_id, test_id in kf.split(X):
    # 훈련 데이터를 사용하여 모델을 작성
    x = X[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier()
    clf.fit(x,y)
    # 테스트 데이터에 모델을 적용
    pred_y = clf.predict(X[test_id])
    # 모델의 정밀도를 계산해서 보존
    score = accuracy_score(Y[test_id], pred_y)
    scores.append(score)

# 모델의 평균정밀도, 표준편차를 확인
scores = np.array(scores)
print(scores.mean(), scores.std())

0.9709866705302812 0.007771887890315759


In [6]:
# Scikit-learn의 로드
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 재현률, 적합률의 확인
#print(confusion_matrix(Y[test_id], pred_y))
print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

0.9629629629629629
0.9904761904761905


In [7]:
# Scikit-learn을 로드
from sklearn.model_selection import GridSearchCV

# 패러메터 범위설정
params = {
    'criterion': ['entropy'],    
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50],
}

# 그리드 서치의 실행조건을 설정
clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, 
                      cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')

# 그리드 서치 실행
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [10]:
print(clf)

DecisionTreeClassifier()


In [8]:
# 가장 높은 정밀도와 패터메터의 조합을 표시
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.9675601274992754
{'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 20}


In [9]:
# 최적의 패터메터를 사용한 모델을 작성
clf_best = tree.DecisionTreeClassifier(
    criterion='entropy', max_depth=10, min_samples_leaf=20)
clf_best.fit(X, Y)

# 영향이 높은 변수를 확인
print(clf_best.feature_importances_)

[0.02323075 0.00555318 0.         0.00735941 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.04861525 0.         0.         0.0120277  0.18375422 0.
 0.02144678 0.         0.66144037 0.         0.03657234 0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
